In [1]:
# -*- encoding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
import tensorflow as tf
import numpy as np
import tflearn
from sklearn import metrics
from tensorflow.contrib import learn
import pandas as pd
# import tensorflow.examples.tutorials.mnist.input_data as input_data
# 获取数据

# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

# log_dir = 'log_dir/'
# max_steps = 1000
# learning_rate = 0.001
# dropout = 0.9


In [3]:
# 划分数据集
def splitData(label,text):

    line_count = len(label)
    rarray=np.random.random(size=line_count)
    train_set = []
    train_tag = []
    test_set = []
    test_tag = []
    for i in range(line_count):
        if rarray[i]<0.8:
            train_set.append(text[i,:])
            train_tag.append(label[i])
        else:
            test_set.append(text[i,:])
            test_tag.append(label[i])
    return (train_set,train_tag,test_set,test_tag)

In [4]:
def trainModel_dense(train_set,train_label,test_set,test_label):
    # 定义深度神经网络
    input_layer = tflearn.input_data(shape=[None, 400])
    
    dense1 = tflearn.fully_connected(input_layer, 400, activation='tanh',regularizer='L2', weight_decay=0.001)
    dropout1 = tflearn.dropout(dense1, 0.8)
    
    dense2 = tflearn.fully_connected(dropout1, 400, activation='tanh',regularizer='L2', weight_decay=0.001)
    dropout2 = tflearn.dropout(dense2, 0.8)
    
    softmax = tflearn.fully_connected(dropout2, 10, activation='softmax')

    # Regression using SGD with learning rate decay and Top-3 accuracy
    sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=3000)
    top_k = tflearn.metrics.Top_k(3)
    net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,loss='categorical_crossentropy')

    # Training
    # tensorboard_verbose=3 可视化
    model = tflearn.DNN(net, tensorboard_verbose=3)
    model.fit(train_set, train_label, n_epoch=30, validation_set=(test_set, test_label),show_metric=True, run_id="dense_model")
    

In [5]:
def trainModel_simple(train_set,train_label,test_set,test_label):
    # 定义深度神经网络
    input_layer = tflearn.input_data(shape=[None, 400])
    
    dense1 = tflearn.fully_connected(input_layer, 64)
    dropout1 = tflearn.dropout(dense1, 0.8)
    
    dense2 = tflearn.fully_connected(dropout1, 64)
    dropout2 = tflearn.dropout(dense2, 0.8)
    
    softmax = tflearn.fully_connected(dropout2, 10, activation='softmax')

    # Regression using SGD with learning rate decay and Top-3 accuracy
    net = tflearn.regression(softmax, optimizer='adam', loss='categorical_crossentropy')

    # Training
    model = tflearn.DNN(net)
    model.fit(train_set, train_label, n_epoch=20, validation_set=(test_set, test_label),show_metric=True, run_id="dense_model")
    

In [6]:
def main():
    
    
    lsiData = "./data/textData_LSI.npy"
    ldaData = "./data/textData_LDA.npy"
    targetData = "./data/target.npy"
    
    # 读入数据 原数据矩阵需要做转置
 
    lsiDataSet = np.load(lsiData).T
    
    ldaDataSet = np.load(ldaData).T
    
    # 读入标签
    target = np.load(targetData)
    
    #切分数据
#     train_set,train_tag,test_set,test_tag = splitData(target,lsiDataSet)
    
    train_set,train_tag,test_set,test_tag = splitData(target,ldaDataSet)
    
    # 把一维的标签做onehot，pd.get_dummies的结果是df,把df转为ndarray (as_matrix())
    train_label = pd.get_dummies(train_tag).as_matrix()
    test_label = pd.get_dummies(test_tag).as_matrix()
    
    # 训练
    trainModel_dense(train_set,train_label,test_set,test_label)    

In [7]:
if __name__ == '__main__':
    main()